<a href="https://colab.research.google.com/github/tayfununal/continuous_CNN/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final Projesi

Our problem is "How to solve the differential equation with using deep methods?

  Especially, we will study on 2 specific problem. These are mass-spring and simple pendulum. We will solve these problem with using 4 different tools. The tools are following:

1.   Solving with neurodiffeq modüle
2.   Solving with Runge-Kutta Method
3.   Solving with deep euler method
4.   Solving with hamiltonian method

Let's look at the differential equation of mass-spring first and then simple pendulum.
## Differential equation of mass-spring


---


The differential equation of mass-spring is
\begin{align*}
                    F(x, t) = \frac{d^2x}{dt^2}+x=0.
\end{align*}
This equation is also second order differential equation. The initial values of the problem are
\begin{align*}
                    x(t=0) = 0 \text{ and } \frac{dx}{dt}|_{t=0} = 1.
\end{align*}

The analytical solution is

\begin{align*}
                    x(t) = sin(t).
\end{align*}


---



## 1) Solving mass-spring with using neurodiffeq 


---



In [ ]:
!pip install neurodiffeq        #Firs we need to install neurodiffeq modüle
import numpy as np
import matplotlib.pyplot as plt


from neurodiffeq import diff      
from neurodiffeq.ode import solve 
from neurodiffeq.conditions import IVP 

In [ ]:
mass_spring = lambda u, t: diff(u, t, order=2) + u
init_val = IVP(t_0=0.0, u_0=0.0, u_0_prime=1.0)

In [ ]:
from neurodiffeq.monitors import Monitor1D

In [ ]:
solution, _ = solve(
    ode=mass_spring, condition=init_val, t_min=0.0, t_max=2*np.pi,
    max_epochs=3000,
    monitor=Monitor1D(t_min=0.0, t_max=2*np.pi, check_every=100)
)

In [ ]:
ts = np.linspace(0, 2*np.pi, 50)
x_net = solution(ts, to_numpy=True)
x_ana = np.sin(ts)

plt.figure()
plt.plot(ts, x_net, label='ANN-based solution')
plt.plot(ts, x_ana, '.', label='analytical solution')
plt.ylabel('x')
plt.xlabel('t')
plt.title('comparing solutions')
plt.legend()
plt.show()

## 2) Solving mass-spring with using Runge-Kutta


---



Our differential equation was
\begin{align*}
                    F(x, t) = \frac{d^2x}{dt^2}+x=0.
\end{align*}
Here, we need to reduce the order of differential equation. For this purpose,

\begin{align*}
                    \text{if }z(t) = x'(t) \text{ then } z'(t) = x''(t).
\end{align*}

As a result, we have 2 equations.

\begin{cases}
                    x'(t) = z(t) \\
                    z'(t) = - x(t)
\end{cases}

Vectorially,  if  $y(t)=[x(t), z(t)]$ , then the equation is  $y′(t)=f(t,y)$  where $f(t,y) = [z(t), -x(t)]$.

The analytical solutions are
\begin{cases}
                    x(t) = sin(t) \\
                    z(t) = cos(t)
\end{cases}

In [ ]:
from scipy.integrate import odeint # odeint module has Runge-kutta methods
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def mass_spring(y, t):
    return np.array([y[1], -y[0]])

y0 = np.array([0 , 1.0])

t = np.linspace(0, 2*np.pi, 51)

sol = odeint(mass_spring, y0, t)

In [ ]:
plt.plot(t, sol[:, 0], 'b', label=r'x(t)')
plt.scatter(t, np.sin(t), c='b', label=r' analytical solution of x(t)')

plt.plot(t, sol[:, 1], 'g', label=r'z(t)')
plt.scatter(t, np.cos(t), c='g', label=r'analytical solution of z(t)')

plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()

## 3) Solving mass-spring with using Deep Euler Method


---



In [ ]:
# Install tfdiffeq if not already installed
!pip install --upgrade -q git+https://github.com/titu1994/tfdiffeq.git

In [ ]:
import tensorflow as tf
tf.keras.backend.set_floatx("float64")
from tfdiffeq import odeint, plot_results
from tfdiffeq.hyper_solvers import HyperEuler
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

tf.executing_eagerly()

In [ ]:
class HyperSolverModule(tf.keras.Model):
  def __init__(self, func_input_dim, hidden_dim=64):
    super().__init__(dtype='float64')
    self.func_input_dim = func_input_dim
    
    # Input dim isnt used (Keras handled it automatically)
    # But for illustration purposes, it is provided
    # Computed as ~ dim(y) + dim(dy) + 1 (for time axis)
    self.input_dim = 2 * func_input_dim + 1
    self.hidden_dim = hidden_dim
    self.output_dim = func_input_dim

    self.g = tf.keras.Sequential([
        tf.keras.layers.Dense(self.hidden_dim),
        tf.keras.layers.PReLU(),
        tf.keras.layers.Dense(self.hidden_dim),
        tf.keras.layers.PReLU(),
        tf.keras.layers.Dense(self.hidden_dim),
        tf.keras.layers.PReLU(),
        tf.keras.layers.Dense(self.output_dim)
    ])
    
  @tf.function
  def call(self, x):
    return self.g(x)

In [ ]:
def mass_spring(t, y0):

  if len(y0.shape) == 1:
    dy_dt = tf.convert_to_tensor([float(y0[1]), -float(y0[0])], dtype=tf.float64)
    return dy_dt
  else:
    dy_dt = tf.convert_to_tensor([float(y0[0,1]), -float(y0[0,0])], dtype=tf.float64)
    return tf.expand_dims(dy_dt, axis=0)

In [ ]:
import math
t = tf.range(0.0, 2*math.pi, 0.01, dtype=tf.float64)
initial_states = tf.convert_to_tensor([0.0, 1.0], dtype=tf.float64)

In [ ]:
%%time

with tf.device('cpu'):
    result = odeint(mass_spring, initial_states, t, rtol=1e-8, atol=1e-8)
print("Finished integrating ! Result shape : ", result.shape)

In [ ]:
plot_results(t, result, labels=['t', 'y'], dependent_vars=True, figsize=(8, 8))
plt.title('mass-spring')

In [ ]:
f = mass_spring
g = HyperSolverModule(func_input_dim=2, hidden_dim=64)
hyper_euler = HyperEuler(f, g)

In [ ]:
epochs = 20
batch_size = 16
pretrain_batches = 100

In [ ]:
%%time
pretrain_batch_data = result[:batch_size, :]

optim = tf.keras.optimizers.Adam(learning_rate=0.01)

for idx in range(pretrain_batches):
    with tf.GradientTape() as tape:
        print(hyper_euler.trajectory(t[:batch_size], initial_states[None, :]))
        output = hyper_euler.trajectory(t[:batch_size], initial_states[None, :])  # SHAPE = [batch_size, 1, 2]
        output = tf.reshape(output, (output.shape[0], -1))  # SHAPE = [batch_size, 2]

        loss = tf.reduce_sum((result[:batch_size] - output) ** 2)

    grads = tape.gradient(loss, hyper_euler.trainable_variables)
    optim.apply_gradients(zip(grads, hyper_euler.trainable_variables))
        
    if (idx + 1) % 10 == 0:
        print(f"Batch {idx + 1}: Loss = {loss.numpy()}")

In [ ]:
%%time 
num_samples = len(result)

sched = tf.keras.experimental.CosineDecay(0.01, decay_steps=num_samples * epochs // batch_size, alpha=1e-4)
optim = tf.keras.optimizers.Adam(learning_rate=sched, clipnorm=10.0)


def train_step(model, t, init_state, y, optimizer):
    with tf.GradientTape() as tape:        
        predictions = hyper_euler.trajectory(t, init_state)  # SHAPE = [batch_size, 1, 2]
        predictions = tf.reshape(predictions, (predictions.shape[0], -1))  # SHAPE = [batch_size, 2]

        loss = tf.reduce_sum((y - predictions) ** 2)
    
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
    return loss, predictions
    

global_steps = 0
for epoch in range(epochs):
  batch_idx = 0
  step = 1
  losses = []

  init_state = initial_states[None, :]

  while batch_idx < num_samples:    
    begin_idx = batch_idx
    end_idx = batch_idx + batch_size

    ground_samples = result[begin_idx: end_idx, :]
    t_samples = t[begin_idx: end_idx]
    init_state = ground_samples[0][None, :]
    
    loss, predictions = train_step(hyper_euler, t_samples, init_state, ground_samples, optim)

    losses.append(loss)
    
    lr = sched(global_steps)

    step += 1
    global_steps += 1
    batch_idx += batch_size

    if step % 20 == 0:
      print(f"Epoch {epoch + 1}, Step {step}: Loss = {loss.numpy()} | LR = {lr.numpy()}")

  average_loss = tf.reduce_mean(tf.stack(losses))
  print(f"End of epoch {epoch + 1}: Average loss = {average_loss.numpy()}")
  print()

In [ ]:
def predict():
    with tf.device('cpu'):
        init_state = initial_states[None, :]  # [[0.0, 1.0,]

        predictions = hyper_euler.trajectory(t, init_state)  # SHAPE = [batch_size, 1, 2]
        predictions = tf.reshape(predictions, (predictions.shape[0], -1))  # SHAPE = [batch_size, 2]
    
    return predictions

In [ ]:
%%time

predictions = predict()

In [ ]:
plot_results(t, predictions, labels=['t', 'y',], dependent_vars=True, figsize=(8, 8))
plt.title('mass-spring')

plot_results(t, result, labels=['t', 'y',], dependent_vars=True, figsize=(8, 8))
plt.title('mass-spring (Ground Truth)')

## Differential equation of simple pendulum


---


The differential equation of simple pendulum is
\begin{align*}
                    F(\theta, t) = \frac{d^2\theta}{dt^2}+sin(\theta)=0.
\end{align*}
This equation is also second order differential equation. The initial values of the problem are
\begin{align*}
                    t_0 = 0\text{ , }\theta_0 = 0.5 \text{ and } \frac{d\theta}{dt}|_{t=0} = 0.
\end{align*}

The analytical solution is

\begin{align*}
                    \theta(t) = \theta_0 cos(t).
\end{align*}


---



## 1) Solving simple pendulum with using neurodiffeq 


---



In [ ]:
!pip install neurodiffeq        #Firs we need to install neurodiffeq modüle
import numpy as np
import matplotlib.pyplot as plt


from neurodiffeq import diff      
from neurodiffeq.ode import solve 
from neurodiffeq.conditions import IVP 

In [ ]:
import torch
simple_pendulum = lambda u, t: diff(u, t, order=2) + torch.sin(u)
init_val = IVP(t_0=0.0, u_0=0.5, u_0_prime=0.0)

In [ ]:
from neurodiffeq.monitors import Monitor1D

In [ ]:
solution, _ = solve(
    ode=simple_pendulum, condition=init_val, t_min=0.0, t_max=2*np.pi,
    max_epochs=3000,
    monitor=Monitor1D(t_min=0.0, t_max=2*np.pi, check_every=100)
)

In [ ]:
ts = np.linspace(0, 2*np.pi, 50)
x_net = solution(ts, to_numpy=True)
x_ana = 0.5*np.cos(ts)

plt.figure()
plt.plot(ts, x_net, label='ANN-based solution')
plt.plot(ts, x_ana, '.', label='analytical solution')
plt.ylabel(r'$\theta$')
plt.xlabel('t')
plt.title('comparing solutions')
plt.legend()
plt.show()

## 2) Solving simple pendulum with using Runge-Kutta


---


Our differential equation was
\begin{align*}
                    F(\theta, t) = \frac{d^2\theta}{dt^2}+sin(\theta)=0.
\end{align*}
Here, we need to reduce the order of differential equation. For this purpose,

\begin{align*}
                    \text{if }z(t) = \theta'(t) \text{ then } z'(t) = \theta''(t).
\end{align*}

As a result, we have 2 equations.

\begin{cases}
                    \theta'(t) = z(t) \\
                    z'(t) = - sin(\theta)
\end{cases}

Vectorially,  if  $y(t)=[\theta(t), z(t)]$ , then the equation is  $y′(t)=f(t,y)$  where $f(t,y) = [z(t), -sin(\theta)]$.

The analytical solutions are
\begin{cases}
                    \theta(t) = sin(t) \\
                    z(t) = cos(t)
\end{cases}

In [ ]:
from scipy.integrate import odeint # odeint module has Runge-kutta methods
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def simple_pendulum(y, t):
    return np.array([y[1], -np.sin(t)])

y0 = np.array([0 , 1.0])

t = np.linspace(0, 2*np.pi, 51)

sol = odeint(simple_pendulum, y0, t)

In [ ]:
plt.plot(t, sol[:, 0], 'b', label=r'$\theta$(t)')
plt.scatter(t, np.sin(t), c='b', label=r' analytical solution of $\theta$(t)')

plt.plot(t, sol[:, 1], 'g', label=r'z(t)')
plt.scatter(t, np.cos(t), c='g', label=r'analytical solution of z(t)')

plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()